In [1]:
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
import random
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 
from selenium.webdriver.common.by import By

## HAPPY FRESH SCRAPING

In [2]:
# Print exception
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [3]:
# Scraping the page
def scrape_detail_page(url,category,collection,store_name):
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.happyfresh_bystore
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]

    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)
    
    # Scrape page
    time.sleep(10)
    result = pd.DataFrame()
    image = 0
    finish = False
    wait = 0
    newHeight = 0
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while (wait < 3):
        newHeight = driver.execute_script("return document.body.scrollHeight")
        time.sleep(5)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        lastHeight = driver.execute_script("return document.body.scrollHeight")
        if(newHeight>=lastHeight):
            wait = wait+1
            time.sleep(10)
            print(wait)
        else:
            wait = 0
            
    time.sleep(5)
    path = driver.find_elements_by_css_selector('.colwrap .products-container > a')
    print('path: '+ str(len(path)))
    for x in range(len(path)):
        try:
            try:
                origin_price = re.sub("\,","",re.sub("Rp","",path[x].find_element_by_css_selector('.prev-price').text))
                discount_price = re.sub("\,","",re.sub("Rp","",path[x].find_element_by_css_selector('.discount').text))
            except:
                origin_price = re.sub("\,","",re.sub("Rp","",path[x].find_elements_by_css_selector('.product-price')[1].text))
                discount_price = origin_price
            name = path[x].find_element_by_css_selector('.product-name').text
            image = path[x].find_element_by_css_selector('.product-image > img').get_attribute('src')
            sku = image
            
            time_check = 0
            while(image is None and time_check < 60):
                time_check = time_check+1
                path[x].find_element_by_css_selector('.product-name').location_once_scrolled_into_view
                time.sleep(1)
                image = path[x].find_element_by_css_selector('.product-image > img').get_attribute('src')
                print("wait_image" + " " + name)
                
            unit = path[x].find_element_by_css_selector('.product-unit').text
            unit_price = re.sub("\,","",re.sub("Rp","",path[x].find_element_by_css_selector('.product-unit-price').text))

            # Return result
            date = datetime.datetime.now().strftime("%Y-%m-%d")
            new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':origin_price, 'discount_price':discount_price, 'image':image,'category':category,'scraping_date':date,'unit':unit,'unit_price':unit_price,'store_name':store_name}])
            result = result.append(new_result)
            
            # save to mongodb
            collection.insert_many(new_result.to_dict('records'))
            
        except:
            PrintException()
            pass
    driver.close()
    return result


In [4]:
# MAIN
happyfresh_category = pd.read_csv('..\\Data\\happyfresh_category.csv',sep=",")
result = pd.DataFrame()
for x in range(41,len(happyfresh_category)):
    category = happyfresh_category.iloc[x]
    print(str(x) + "/" + str(len(happyfresh_category)) + " " + category['category'] + " - " + category['sub_category'])
    
    store_name = 'transmart-carrefour-mt-haryono/'
    url = "https://www.happyfresh.id/"+store_name+"/"+category['link']
    collection = category['category']
    
    result = scrape_detail_page(url,category['sub_category'],collection,store_name)
    print(result.head())

41/82 frozen - Frozen Meat Seafood
Connected successfully!!!
1
2
3
path: 14
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku
wait_image Panorama Udang Masak Beku

wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image Campina Es Krim Chocolate Cunks
wait_image 

1
2
3
path: 1
          category discount_price  \
0  Bakery Desserts          28300   

                                               image  \
0  https://cdn.happyfresh.com/spree/images/attach...   

                            name origin_price scraping_date  \
0  Edo Mini Pau Isi Kacang Merah        28300    2019-09-19   

                                                 sku  \
0  https://cdn.happyfresh.com/spree/images/attach...   

                        store_name      unit unit_price  
0  transmart-carrefour-mt-haryono/  8 x 25 g      14150  
48/82 bakery - Buns Rolls
Connected successfully!!!
1
2
3
path: 1
     category discount_price  \
0  Buns Rolls          10400   

                                               image                   name  \
0  https://cdn.happyfresh.com/spree/images/attach...  Sari Roti Roti Burger   

  origin_price scraping_date  \
0        10400    2019-09-19   

                                                 sku  \
0  https://cdn.happyfresh.com/s

1
2
3
path: 50
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image SoKlin Lantai Spa Aromatik Floral Lavender
wait_image So

1
2
3
path: 50
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_image Kispray Pelicin Pakaian Fine Perfume Glamorous...
wait_imag

...

wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gigi Pencegah Gigi Berlubang
wait_image Colgate Pasta Gig

1
2
3
path: 50
wait_image Enervon Active Multivitamin dan Mineral Suplemen...
wait_image Neurobion Vitamin B1,B6,B12
             category discount_price  \
0  Health Supplements          46200   
0  Health Supplements         157000   
0  Health Supplements          49100   
0  Health Supplements           6000   
0  Health Supplements           2300   

                                               image  \
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   

                                      name origin_price scraping_date  \
0          Enervon-C Suplemen Multivitamin        46200    2019-09-19   
0            BRAND'S Minuman Saripati Ayam       157000    2019-09-19   
0              CDR Calcium-D-Redoxon Jeruk        54500    2019-09-19   
0  Enervon-c 

1
2
3
path: 50
wait_image Mitu Tisu Basah Bayi Antiseptik
wait_image Cussons Tisu Basah Bayi Anti Bakteri
        category discount_price  \
0  Diapers Wipes          21700   
0  Diapers Wipes          21700   
0  Diapers Wipes          22400   
0  Diapers Wipes          20000   
0  Diapers Wipes          22400   

                                               image  \
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   

                                                name origin_price  \
0           Cussons Tisu Basah Bayi Lembut dan Halus        21700   
0          Cussons Tisu Basah Bayi Fresh and Nourish        21700   
0  Mitu Tisu Ganti Popok Bayi dengan Chrysanthemu...        22400   
0                  Paseo Tisu Basah Bayi Non Alkohol        20000   
0

1
2
3
path: 50
wait_image Friskies Makanan Kucing Seafood Sensations 450g
wait_image Friskies Daging Bakar
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu dengan...
wait_image Purrfect Daging Tuna Merah dalam Kaldu den

1
2
3
path: 50
wait_image Dji Sam Soe Super Premium Rokok Kretek
wait_image Clas Mild Rokok
     category discount_price  \
0  Cigarettes          18900   
0  Cigarettes          29500   
0  Cigarettes          29300   
0  Cigarettes          23100   
0  Cigarettes          30200   

                                               image  \
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   
0  https://cdn.happyfresh.com/spree/images/attach...   

                                             name origin_price scraping_date  \
0  Gudang Garam International Rokok Kretek Filter        18900    2019-09-19   
0                 Marlboro Ice Blast Rokok Methol        29500    2019-09-19   
0               Marlboro Gold Lights Rokok Filter        29300    2019-09-19   
0             Djarum L.A. Ice Rokok Kretek Filter        2

In [34]:
x

41

## After Scraping

In [4]:
def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db_out = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db_out[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if '_id' in df.columns:
        del df['_id']
    
    if db == 'visenze_tagging':
        del df['created_date'], df['metadata'], df['box']
    return df

In [9]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

dbname = 'happyfresh_bystore'
store_name = 'super-indo-kincan'
mydb = myclient[dbname]
result = pd.DataFrame()

#list the collections
for coll in mydb.collection_names():
    data = read_mongo(dbname,coll)
    data['collection'] = coll
    data = data[data.store_name==store_name]
    result = result.append(data)

result = result.drop_duplicates(['name','image'])
filename = "HAPPYFRESH_"+store_name+"_"+str(datetime.datetime.now().strftime("%Y%m%d"))+'.csv'
result.to_csv('..\\Result\\'+filename,index=False)

In [7]:
coll

'happyfresh'